### Read data

In [1]:
import pandas as pd


opencsv = pd.read_csv(r"https://raw.githubusercontent.com/notp1ssed/ProjetoFinal-PosGraduacaoEngenhariaFinanceira/main/df_weekly.csv", sep=",")
opencsv.set_index('Date', inplace=True)
opencsv.index = pd.to_datetime(opencsv.index)
opencsv = opencsv.asfreq(pd.infer_freq(opencsv.index))
opencsv = opencsv.astype(float)
opencsv = opencsv["BTC_USD"]

In [3]:
opencsv

Date
2010-12-10        0.204
2010-12-17        0.240
2010-12-24        0.248
2010-12-31        0.300
2011-01-07        0.320
                ...    
2024-05-31    67482.250
2024-06-07    69330.510
2024-06-14    66006.130
2024-06-21    64119.340
2024-06-28    60324.300
Freq: W-FRI, Name: BTC_USD, Length: 708, dtype: float64

### Full Time Frame Halving dividers

In [45]:
import plotly.graph_objects as go


## ciclos
halving12 = "2012-11-28"
halving16 = "2016-07-09"
halving20 = "2020-05-11"
halving24 = "2024-04-19"

halvings_dates = [halving12, halving16, halving20, halving24]

# Fazer gráfico vazio
fig = go.Figure()

# Adicionar as linhas (uma a uma)
fig = fig.add_trace(go.Scatter(x=opencsv.index, y=opencsv, name='BTC-USD'))

# Adicionar linhas verticais para sinalizar eventos em datas específicas

for date in halvings_dates:
    fig.add_shape(
    dict(
        type="line",
        x0=date, x1=date,  # Data de cada halving
        y0=0, y1=1,  # Usar 0 e 1 para cobrir todo o eixo y
        xref='x',
        yref='paper',
        line=dict(
            color="Red",
            width=2,
            dash="dot",),)
    )
    
    # Adicionar anotações para as linhas verticais
    fig.add_annotation(
        x=date,  # Data de cada halving
        y=0.1,
        xref="x",
        yref="paper",
        text=date,
        showarrow=False,
        yshift=10,
        font=dict(color="Black")
    )


# Acrescentar tema
fig = fig.update_layout(template='simple_white',
                   title='Bitcoin halving cycles',
                   height=700,
                   width=1200,
                   hovermode="x unified",
                   showlegend=True,
                   legend=dict(yanchor="bottom", y=1, xanchor="center",x=0.5, orientation='h'),
                   xaxis=dict(showgrid=False, zeroline=False, showline=False, mirror=True),
                   yaxis=dict(title='Preço ($)', ticksuffix=' $', showgrid=False, zeroline=True, showline=False, mirror=True, type='log'),
                   ).show()

### Splitted Halving cycles

In [46]:
## ciclos
halving12 = "2012-11-28"
halving16 = "2016-07-09"
halving20 = "2020-05-11"
halving24 = "2024-04-19"


prep_df = opencsv.copy()

ciclo1 = prep_df.loc[:halving12]
ciclo2 = prep_df.loc[halving12:halving16]
ciclo3 = prep_df.loc[halving16:halving20]
ciclo4 = prep_df.loc[halving20:halving24]
ciclo5 = prep_df.loc[halving24:]

In [47]:
import plotly.graph_objects as go

halving_cycles = pd.DataFrame()
halving_cycles['ciclo1'] = ciclo1.reset_index(drop=True)
halving_cycles['ciclo2'] = ciclo2.reset_index(drop=True)
halving_cycles['ciclo3'] = ciclo3.reset_index(drop=True)
halving_cycles['ciclo4'] = ciclo4.reset_index(drop=True)
halving_cycles['ciclo5'] = ciclo5.reset_index(drop=True)

In [48]:
halving_cycles

,ciclo1,ciclo2,ciclo3,ciclo4,ciclo5
0,0.204,12.57,664.20,9311.69,63848.26
1,0.240,13.50,651.11,9170.74,63750.63
2,0.248,13.60,655.43,9424.62,62917.45
3,0.300,13.50,574.66,9622.02,60790.82
4,0.320,13.50,587.14,9464.23,67051.35
...,...,...,...,...,...
98,10.190,380.48,7530.55,46299.75,NaN
99,10.470,355.10,7627.52,42273.38,NaN
100,10.820,337.87,6396.71,40563.69,NaN
101,11.750,341.87,6051.47,39716.04,NaN


In [49]:
import plotly.graph_objects as go

def normalize(df):
    return (df / df.iloc[0]) * 100

halvings_normalized = normalize(halving_cycles)

def comparative_graph(df):
    # Fazer com que as séries temporais comecem do mesmo ponto
    df_normalized = normalize(df)

    performance_compare = go.Figure()
    # Iterate over the tickers list
    for ticker in df.columns:
        performance_compare.add_trace(go.Scatter(name=f'{ticker}', x=df_normalized.index, y=df_normalized[ticker]))

    # Acrescentar tema
    performance_compare.update_layout(template='simple_white',
                   title='Retornos ciclos halving Bitcoin',
                   height=700,
                   width=1200,
                   hovermode="x unified",
                   showlegend=True,
                   legend=dict(yanchor="bottom", y=1, xanchor="center",x=0.5, orientation='h'),
                   xaxis=dict(title='Nr dias após halving', showgrid=False, zeroline=False, showline=False, mirror=True),
                   yaxis=dict(title='Valorização (%)', ticksuffix=' %', showgrid=False, zeroline=True, showline=False, mirror=True, type='log'),
                   ).show()

In [43]:
comparative_graph(halvings_normalized)